In [1]:
import os
from shutil import copy2
import csv

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tensorflow_hub as hub
import pandas as pd

In [4]:
tf.__version__

'2.3.1'

In [5]:
# Increase precision of presented data for better side-by-side comparison
pd.set_option("display.precision", 8)

In [6]:
data_root = "../dataset/AWEDataset/awe-train"

In [7]:
#https://medium.com/analytics-vidhya/how-to-do-image-classification-on-custom-dataset-using-tensorflow-52309666498e

In [8]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = "/".join([data_root, "train"])
VALIDATE_DATA_DIR = "/".join([data_root, "val"])

datagen_kwargs = dict(rescale=1./255)


# validation set
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    VALIDATE_DATA_DIR, 
    subset="training", 
    shuffle=True,
    target_size=IMAGE_SHAPE)

# training set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=15,
      brightness_range=[0.5, 1.5],
      horizontal_flip=False,
      vertical_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="training", 
    shuffle=True,
    target_size=IMAGE_SHAPE)

Found 100 images belonging to 100 classes.
Found 800 images belonging to 100 classes.


In [9]:
for image_batch, label_batch in train_generator:
      break
image_batch.shape, label_batch.shape

((32, 224, 224, 3), (32, 100))

In [10]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
      f.write(labels)

print (valid_generator.class_indices)
labels = '\n'.join(sorted(valid_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
      f.write(labels)

{'001': 0, '002': 1, '003': 2, '004': 3, '005': 4, '006': 5, '007': 6, '008': 7, '009': 8, '010': 9, '011': 10, '012': 11, '013': 12, '014': 13, '015': 14, '016': 15, '017': 16, '018': 17, '019': 18, '020': 19, '021': 20, '022': 21, '023': 22, '024': 23, '025': 24, '026': 25, '027': 26, '028': 27, '029': 28, '030': 29, '031': 30, '032': 31, '033': 32, '034': 33, '035': 34, '036': 35, '037': 36, '038': 37, '039': 38, '040': 39, '041': 40, '042': 41, '043': 42, '044': 43, '045': 44, '046': 45, '047': 46, '048': 47, '049': 48, '050': 49, '051': 50, '052': 51, '053': 52, '054': 53, '055': 54, '056': 55, '057': 56, '058': 57, '059': 58, '060': 59, '061': 60, '062': 61, '063': 62, '064': 63, '065': 64, '066': 65, '067': 66, '068': 67, '069': 68, '070': 69, '071': 70, '072': 71, '073': 72, '074': 73, '075': 74, '076': 75, '077': 76, '078': 77, '079': 78, '080': 79, '081': 80, '082': 81, '083': 82, '084': 83, '085': 84, '086': 85, '087': 86, '088': 87, '089': 88, '090': 89, '091': 90, '092': 9

In [11]:
IMAGE_SIZE = 224

In [12]:
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               128100    
Total params: 2,386,084
Trainable params: 128,100
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [14]:
model.compile(
  optimizer=optimizer,
  loss='categorical_crossentropy',
  metrics=['acc'])

In [15]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator, 
    epochs=1000,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

Epoch 1/1000
25/25 [==============================] - 9s 366ms/step - loss: 5.7487 - acc: 0.0063 - val_loss: 4.7442 - val_acc: 0.0600
Epoch 2/1000
25/25 [==============================] - 9s 349ms/step - loss: 4.6969 - acc: 0.0463 - val_loss: 4.4666 - val_acc: 0.0600
Epoch 3/1000
25/25 [==============================] - 9s 348ms/step - loss: 4.0703 - acc: 0.1050 - val_loss: 4.2867 - val_acc: 0.1200
Epoch 4/1000
25/25 [==============================] - 9s 351ms/step - loss: 3.6689 - acc: 0.1625 - val_loss: 4.1818 - val_acc: 0.1100
Epoch 5/1000
25/25 [==============================] - 9s 350ms/step - loss: 3.2180 - acc: 0.2275 - val_loss: 4.0721 - val_acc: 0.1400
Epoch 6/1000
25/25 [==============================] - 9s 351ms/step - loss: 2.9900 - acc: 0.2750 - val_loss: 3.9982 - val_acc: 0.1500
Epoch 7/1000
25/25 [==============================] - 9s 350ms/step - loss: 2.7003 - acc: 0.3425 - val_loss: 3.9175 - val_acc: 0.2000
Epoch 8/1000
25/25 [==============================] - 9s 351ms

KeyboardInterrupt: 